In [78]:
import requests
import os
import sys
import datetime
import pytz
import yaml
import pandas as pd

In [79]:
if '../' not in sys.path:
    sys.path.append('../')
 

In [80]:
   from flexlab.db_layer.db_interface import DB_Interface

In [81]:
def send_simple_message(api_key, recipients, message):
    return requests.post(
        "https://api.mailgun.net/v3/sandbox244efac4658643d69658912e514bddb1.mailgun.org/messages",
        auth=("api", api_key),
        data={"from": "Excited User <akprakash@lbl.gov>",
            "to": recipients,
            "subject": "Missing data alert",
            "text": message})

In [82]:
if __name__ == "__main__":

    with open("db_interface_config_template_metrics.yaml") as fp:
    #with open("alerts_config.yaml") as fp:
    
    # original file: "alerts_config.yaml" 
        config = yaml.safe_load(fp)

    api_key = config.get('api_key')
    recipients = config.get('recipients')

    tz_local = pytz.timezone("US/Pacific")
    tz_utc = pytz.timezone("UTC")

    db = DB_Interface(config_file='db_interface_config_template_metrics.yaml')
    #db = DB_Interface(config_file='db_interface_config_alerts.yaml')
    
    # original file: "db_interface_config_alerts.yaml"

    et_utc = tz_utc.localize(datetime.datetime.utcnow())
    et_local = et_utc.astimezone(tz_local)
    st_local = et_local - datetime.timedelta(minutes=30)
    
    # Weiping changes the next 2 lines
    
    #st_local = et_local - datetime.timedelta(hours=67)
    #et_local = et_local - datetime.timedelta(hours=65)

    
    error_cols_dict = {'1a': [], '1b': []}
    
    cell_1a_df=db.get_data(st=st_local, et=et_local, cell='1a') # full cell 1A data
    cell_1b_df=db.get_data(st=st_local, et=et_local, cell='1b') # full cell 1B data
    

    first_datapoint='1A-ASAM-1'            # name of first datapoint to be compared
    second_datapoint='1B-ASAM-1'         # name of second datapoint to be compared
    df1=cell_1a_df[first_datapoint] 
    df2=cell_1b_df[second_datapoint] 
    df=pd.concat([df1, df2], axis=1)
    
    
    df_mean=df.mean(axis=0)
    error_tolerance_percentage=5      # in unit of %
    percent_difference=abs((df_mean.iloc[0]-df_mean.iloc[1])/df_mean.iloc[0]*100)

    if percent_difference>error_tolerance_percentage:
        error=True
    else:
        error=False

    msg = "The following columns have dissimilar values for more than 30 minutes: "+first_datapoint+', '+second_datapoint+'.'
    #msg += '\n' + ', '.join(error_cols) + '\n'

    if error:
        print(msg)
        print(recipients)
        rsp = send_simple_message(api_key=api_key, recipients=recipients, message=msg)

        if rsp.status_code == 200:
            print("successfully sent alert")
        else:
            print("error sending alert: "+ str(rsp.content))
    else:
        print("no error columns")

The following columns have dissimilar values for more than 30 minutes: 1A-ASAM-1, 1B-ASAM-1.
None
error sending alert: b'Forbidden'


In [83]:
print(percent_difference)

8.872442997347452


# Example datapoints

In [84]:
cell_1a_df=db.get_data(st=st_local, et=et_local, cell='1a') # cell can be '1a' or '1b'


In [85]:
cell_1b_df=db.get_data(st=st_local, et=et_local, cell='1b') # cell can be '1a' or '1b'


In [86]:
df3=cell_1a_df['1A-ASAM-1']
print(df3.tail(15))

time
2021-09-21 21:30:00    430.435000
2021-09-21 21:45:00    429.176000
2021-09-21 22:00:00    429.832000
2021-09-21 22:15:00    429.722667
2021-09-21 22:30:00    429.232667
2021-09-21 22:45:00    428.613333
2021-09-21 23:00:00    429.202000
2021-09-21 23:15:00    429.821333
2021-09-21 23:30:00    430.828462
Freq: 15T, Name: 1A-ASAM-1, dtype: float64


In [87]:
df4=cell_1b_df['1B-ASAM-1']
print(df4.tail(15))

time
2021-09-21 21:30:00    468.970000
2021-09-21 21:45:00    471.356000
2021-09-21 22:00:00    468.695333
2021-09-21 22:15:00    467.460000
2021-09-21 22:30:00    465.758000
2021-09-21 22:45:00    465.891333
2021-09-21 23:00:00    466.706667
2021-09-21 23:15:00    466.816000
2021-09-21 23:30:00    468.295385
Freq: 15T, Name: 1B-ASAM-1, dtype: float64


In [88]:
df0=pd.concat([df3, df4], axis=1)
print(df0)

                      1A-ASAM-1   1B-ASAM-1
time                                       
2021-09-21 21:30:00  430.435000  468.970000
2021-09-21 21:45:00  429.176000  471.356000
2021-09-21 22:00:00  429.832000  468.695333
2021-09-21 22:15:00  429.722667  467.460000
2021-09-21 22:30:00  429.232667  465.758000
2021-09-21 22:45:00  428.613333  465.891333
2021-09-21 23:00:00  429.202000  466.706667
2021-09-21 23:15:00  429.821333  466.816000
2021-09-21 23:30:00  430.828462  468.295385
